[Habr](https://habrahabr.ru/company/ods/blog/327242/)
# Временные ряды

## Движемся, сглаживаем, оцениваем
#### Rolling window
* скользящее среднее [DataFrame.rolling(window).mean()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html)
* оно же, но с весами (обычно более близким событиям -- больше)

#### Экспоненциальное сглаживание, модель Хольта-Винтерса
* простое эксп. сглаживание: используем все прошлое, но с уменьшающимися экспоненциально весами $\hat{y}_{t} = \alpha \cdot y_t + (1-\alpha) \cdot \hat y_{t-1}$, где $\alpha$ -- сглаживающий фактор (чем больше, тем больше используется прошлое)
* двойное эксп. сглаживание: пытаемся предсказывать отдельно уровень (level, intercept) и тренд (trend, slope). Подбирать уже два параметры сглаживания
* тройное эксп. сглаживание: добавляем сезонность (с фиксированной длиной сезона)

Итого перешли от предсказывания 1 значения через 2 к бесконечному числу

#### Кросс-валидация на временных рядах, подбор параметров
Оценка ошибки как обычно -- выбираем функцию потерь, считаем градиент, двигаемся к лучшим параметрам.

Проблема в кросс-валидации: данные нельзя просто перемешать. Рабочий вариант -- "cross-validation on a rolling basis" (кросс-валидация на скользящем окне). Идея -- обучаемся на кусочке, предсказываем следующие n, считаемся. Добавляем эти n к обучению, повторяем

## Эконометрический подход
#### Стационарность, единичные корни
Стационарность: статистические характеристики не меняются
* постоянство матожидания
* постоянство дисперсии (гомоскедастичность)
* независимость ковариационной функции от времени (только от расстояния между значениями)
К сожалению, это редко бывает в реальном мире, но к этому можно стремиться

Проверка стационарности: `sm.tsa.stattools.adfuller(x)[1]`, [тест Дики Фуллера](https://ru.wikipedia.org/wiki/Тест_Дики_—_Фуллера). H0 -- ряд стационарен

#### Борьба с нестационарностью
Бороться с нестационарностью можно множеством способов — разностями различного порядка, выделением тренда и сезонности, сглаживаниями и преобразованиями, например, Бокса-Кокса или логарифмированием.

[Пример про Бокса-Кокса]

Чо-та лень




## Линейные и не очень модели на временных рядах
Давайте просто фич повыделяем и обучим что-нибудь.

#### Feature extraction
[Дата-время](https://habrahabr.ru/company/ods/blog/325422/#data-i-vremya)
* one-hot encoding дней недели, месяцев etc. Выходные.
* время -- сложно. Плохой полный порядок, но не хотим терять инфу о близости -- можно на окружность переложить

Еще хорошо работает средний для категории (по трейну!). Также максимальное/минимальное значение, наблюдавшееся в скользящем по ряду окне, медианы, число пиков, взвешенные дисперсии и многое другое. Автоматически этим занимается уже упоминавшаяся в курсе библиотека [библиотека tsfresh](https://github.com/blue-yonder/tsfresh).

#### Линейная регрессия

Обучим на получившихся данных простую линейную регрессию. При этом **лаги будем брать начиная с двенадцатого**, таким образом модель будет способна строить предсказания на 12 часов вперёд, имея фактические наблюдения за предыдущие пол дня.

In [1]:
def prepareData(data, lag_start=5, lag_end=20, test_size=0.15):

    data = pd.DataFrame(data.copy())
    data.columns = ["y"]

    # считаем индекс в датафрейме, после которого начинается тестовыый отрезок
    test_index = int(len(data)*(1-test_size))

    # добавляем лаги исходного ряда в качестве признаков
    for i in range(lag_start, lag_end):
        data["lag_{}".format(i)] = data.y.shift(i)

    data.index = data.index.to_datetime()
    data["hour"] = data.index.hour
    data["weekday"] = data.index.weekday
    data['is_weekend'] = data.weekday.isin([5,6])*1

    # считаем средние только по тренировочной части, чтобы избежать лика
    data['weekday_average'] = map(code_mean(data[:test_index], 'weekday', "y").get, data.weekday)
    data["hour_average"] = map(code_mean(data[:test_index], 'hour', "y").get, data.hour)

    # выкидываем закодированные средними признаки 
    data.drop(["hour", "weekday"], axis=1, inplace=True)

    data = data.dropna()
    data = data.reset_index(drop=True)

    # разбиваем весь датасет на тренировочную и тестовую выборку
    X_train = data.loc[:test_index].drop(["y"], axis=1)
    y_train = data.loc[:test_index]["y"]
    X_test = data.loc[test_index:].drop(["y"], axis=1)
    y_test = data.loc[test_index:]["y"]

    return X_train, X_test, y_train, y_test

In [ ]:
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = prepareData(dataset.Users, test_size=0.3, lag_start=12, lag_end=48)
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X_test)
plt.figure(figsize=(15, 7))
plt.plot(prediction, "r", label="prediction")
plt.plot(y_test.values, label="actual")
plt.legend(loc="best")
plt.title("Linear regression\n Mean absolute error {} users".format(round(mean_absolute_error(prediction, y_test))))
plt.grid(True);

#### Кросс-валидация

In [ ]:
def performTimeSeriesCV(X_train, y_train, number_folds, model, metrics):
    print('Size train set: {}'.format(X_train.shape))

    k = int(np.floor(float(X_train.shape[0]) / number_folds))
    print('Size of each fold: {}'.format(k))

    errors = np.zeros(number_folds-1)

    # loop from the first 2 folds to the total number of folds    
    for i in range(2, number_folds + 1):
        print('')
        split = float(i-1)/i
        print('Splitting the first ' + str(i) + ' chunks at ' + str(i-1) + '/' + str(i) )

        X = X_train[:(k*i)]
        y = y_train[:(k*i)]
        print('Size of train + test: {}'.format(X.shape)) # the size of the dataframe is going to be k*i

        index = int(np.floor(X.shape[0] * split))

        # folds used to train the model        
        X_trainFolds = X[:index]        
        y_trainFolds = y[:index]

        # fold used to test the model
        X_testFold = X[(index + 1):]
        y_testFold = y[(index + 1):]

        model.fit(X_trainFolds, y_trainFolds)
        errors[i-2] = metrics(model.predict(X_testFold), y_testFold)

    # the function returns the mean of the errors on the n-1 folds    
    return errors.mean()